In [1]:
pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /Users/yutongyin/opt/anaconda3/lib/python3.7/site-packages (0.23.2)
Note: you may need to restart the kernel to use updated packages.


In [10]:
from sklearn.model_selection import KFold # import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
import numpy as np
import pandas as pd
from sklearn.metrics import explained_variance_score
# Import function to create training and test set splits
from sklearn.model_selection import train_test_split
# Import function to automatically create polynomial features! 
from sklearn.preprocessing import PolynomialFeatures
# Import Linear Regression and a regularized regression function
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
# Finally, import function to make a machine learning pipeline
from sklearn.pipeline import make_pipeline
from sklearn import metrics

In [11]:
ML_data= pd.read_excel('ML training data_draft1.xlsx')
print(ML_data)

         #Set_name      ORF  gRNA length Gene_name Chromosome  ORF_TSS  \
0     broad_tiling  YHR007C           20     ERG11       VIII   121825   
1     broad_tiling  YHR007C           20     ERG11       VIII   121825   
2     broad_tiling  YHR007C           20     ERG11       VIII   121825   
3     broad_tiling  YHR007C           20     ERG11       VIII   121825   
4     broad_tiling  YHR007C           20     ERG11       VIII   121825   
..             ...      ...          ...       ...        ...      ...   
393  gene_tiling20  YIL143C           20      SSL2         IX    83078   
394  gene_tiling20  YIL143C           20      SSL2         IX    83078   
395  gene_tiling20  YIL143C           20      SSL2         IX    83078   
396  gene_tiling20  YIL143C           20      SSL2         IX    83078   
397  gene_tiling20  YIL143C           20      SSL2         IX    83078   

            Guide_name  Specificity_sequence activation/repression  \
0    ERG11-121332-20bp  ATTTGTCTTCAACGCTA

In [12]:
ML_data.head()

,#Set_name,ORF,gRNA length,Gene_name,Chromosome,ORF_TSS,Guide_name,Specificity_sequence,activation/repression,Aimed_strand,ORF_strand,PAM_loc,Guide_midpoint_loc,TSS_distance,GC_content,chromatin_accessibility,nucleosome_occupancy,Score,var(score)
0,broad_tiling,YHR007C,20,ERG11,VIII,121825,ERG11-121332-20bp,ATTTGTCTTCAACGCTAAGT,repression,template,-,121332,121345,480,35.0,2.871765,0.316207,0.406,0.00463
1,broad_tiling,YHR007C,20,ERG11,VIII,121825,ERG11-121361-20bp,ACTGTGTATTTAGGACCAAA,repression,template,-,121361,121374,451,35.0,6.342353,0.305063,0.038,0.02190
2,broad_tiling,YHR007C,20,ERG11,VIII,121825,ERG11-121362-20bp,GACTGTGTATTTAGGACCAA,repression,template,-,121362,121375,450,40.0,6.361961,0.282501,0.791,0.00730
3,broad_tiling,YHR007C,20,ERG11,VIII,121825,ERG11-121370-20bp,AGAGTCATGACTGTGTATTT,repression,template,-,121370,121383,442,35.0,6.028627,0.203691,0.417,0.00453
4,broad_tiling,YHR007C,20,ERG11,VIII,121825,ERG11-121421-20bp,GAAGAATGTCAAAAGAAATA,repression,template,-,121421,121434,391,25.0,1.656078,-0.028593,0.423,0.02810


# KFOLD CV

In [16]:
X=ML_data.iloc[:,13:17].values
y=ML_data.iloc[:, 17].values
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.2, random_state=0)

In [17]:
#KFOLD
kf = KFold(n_splits=5, random_state=2, shuffle=True) # Define the split - into 5 folds 
print(kf) 

KFold(n_splits=5, random_state=2, shuffle=True)


In [18]:
reg = linear_model.LinearRegression()
model = reg.fit(X_train, y_train)

In [19]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train,y_train)
    print(model.coef_)
    print(model.intercept_)
    return model.score(X_test,y_test)

In [20]:
scores_linearreg =[]
equation_linearreg=[]


for train_index, test_index in kf.split(X):
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]
   reg = linear_model.LinearRegression()
   model = reg.fit(X_train, y_train)
   y_test_pred= model.predict(X_test)
   mse = mean_squared_error(y_test, y_test_pred)
   scores_linearreg.append(mse)
   equation_linearreg.append(get_score(LinearRegression(),X_train, X_test, y_train, y_test))

[-3.81947096e-04  2.81544028e-02 -4.29975373e-04  1.43297313e+00]
-2.5523946022824995
[-6.71565081e-04  2.12627880e-02 -1.82131932e-03  1.61788111e+00]
-2.085970626600925
[-6.04415415e-04  3.23350269e-02 -2.60793899e-04  1.38522067e+00]
-2.7238930047222665
[-3.51023056e-04  2.49549616e-02 -3.45253306e-03  1.30319328e+00]
-2.34470071038822
[-1.87301299e-04  1.00925079e-03  1.59057608e-03  1.54779786e+00]
-1.354921401584575


In [21]:
scores_linearreg

[6.55351411301942,
 8.329232956232016,
 5.625067842842306,
 7.180343405451373,
 8.25832042124973]

In [22]:
avg_mse =np.average(scores_linearreg)

In [23]:
avg_mse

7.189295747758969

In [24]:
equation_linearreg

[0.13059738026856138,
 0.08103125565882985,
 0.13982341065511183,
 0.20070269566151033,
 0.128165657635266]

In [25]:
avg_accuracy =np.average(equation_linearreg)

In [26]:
avg_accuracy

0.13606407997585587

In [27]:
#testing polynomial relationship with CV

for degree in range(1,5):
    
    poly = make_pipeline(PolynomialFeatures(degree), Ridge())
    scores_polylinearreg =[]

    for train_index, test_index in kf.split(X):
       X_train, X_test = X[train_index], X[test_index]
       y_train, y_test = y[train_index], y[test_index]
       model = poly.fit(X_train, y_train)
       y_test_pred= model.predict(X_test)
       mse = mean_squared_error(y_test, y_test_pred)
       scores_polylinearreg.append(mse)
        
    avg =np.average(scores_polylinearreg)
    print("For degree "+str(degree)+" found average mse value: "+str(avg))

For degree 1 found average mse value: 7.188979959459809
For degree 2 found average mse value: 6.2838409264711474
For degree 3 found average mse value: 7.284892696467584
For degree 4 found average mse value: 47.19053047040666


/Users/yutongyin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.5387e-18): result may not be accurate.
  overwrite_a=True).T
/Users/yutongyin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.61739e-18): result may not be accurate.
  overwrite_a=True).T
/Users/yutongyin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.567e-18): result may not be accurate.
  overwrite_a=True).T
/Users/yutongyin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.52907e-18): result may not be accurate.
  overwrite_a=True).T
/Users/yutongyin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.7726e-18): result may not be accurate.
  over

In [28]:
#testing polynomial relationship with CV

for degree in range(1,5):
    
    poly = make_pipeline(PolynomialFeatures(degree), Ridge())
    scores_polylinearreg =[]

    for train_index, test_index in kf.split(X):
       X_train, X_test = X[train_index], X[test_index]
       y_train, y_test = y[train_index], y[test_index]
       model = poly.fit(X_train, y_train)
       y_test_pred= model.predict(X_test)
       predscore = explained_variance_score(y_test, y_test_pred)
       scores_polylinearreg.append(predscore)
    
    avg =np.average(scores_polylinearreg)
    print("For degree "+str(degree)+" found average accuracy score : "+str(avg))

For degree 1 found average accuracy score : 0.1395663872323873
For degree 2 found average accuracy score : 0.2606758141574087
For degree 3 found average accuracy score : 0.154872660116726
For degree 4 found average accuracy score : -4.168340001770414


/Users/yutongyin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.5387e-18): result may not be accurate.
  overwrite_a=True).T
/Users/yutongyin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.61739e-18): result may not be accurate.
  overwrite_a=True).T
/Users/yutongyin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.567e-18): result may not be accurate.
  overwrite_a=True).T
/Users/yutongyin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.52907e-18): result may not be accurate.
  overwrite_a=True).T
/Users/yutongyin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.7726e-18): result may not be accurate.
  over